In [1]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('smsspamcollection.tsv', sep='\t')

In [5]:
df.head()

,label,message
0,ham,I've been searching for the right words to tha...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,"Nah I don't think he goes to usf, he lives aro..."
3,ham,Even my brother is not like to speak with me. ...
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!


In [7]:
df.isnull().sum()

label      0
message    0
dtype: int64

In [8]:
len(df)

5568

In [9]:
df['label'].unique()

array(['ham', 'spam'], dtype=object)

In [10]:
df['label'].value_counts()

ham     4822
spam     746
Name: label, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X = df['message']

In [15]:
y = df['label']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

In [18]:
count_vect = CountVectorizer()

In [19]:
# Fit Vectorizer to the data

X_train_counts = count_vect.fit_transform(X_train)

In [20]:
X_train_counts

<3730x7118 sparse matrix of type '<class 'numpy.int64'>'
	with 50072 stored elements in Compressed Sparse Row format>

In [21]:
X_train.shape

(3730,)

In [22]:
X_train_counts.shape

(3730, 7118)

### Tf-Idf

In [24]:
from sklearn.feature_extraction.text import TfidfTransformer

In [25]:
tfidf = TfidfTransformer()

In [26]:
X_train_tfidf = tfidf.fit_transform(X_train_counts)

In [53]:
X_train_tfidf.shape

(3730, 7118)

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [33]:
vectorizer = TfidfVectorizer()

In [34]:
X_train_tfidf = vectorizer.fit_transform(X_train)

In [35]:
from sklearn.svm import LinearSVC

In [36]:
clf = LinearSVC()

In [38]:
clf.fit(X_train_tfidf, y_train)

LinearSVC()

#### Using Pipeline

In [39]:
from sklearn.pipeline import Pipeline

In [40]:
text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])

In [41]:
text_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [42]:
pred = text_clf.predict(X_test)

In [43]:
from sklearn.metrics import confusion_matrix, classification_report

In [44]:
print(confusion_matrix(y_test, pred))

[[1605    3]
 [  22  208]]


In [45]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1608
        spam       0.99      0.90      0.94       230

    accuracy                           0.99      1838
   macro avg       0.99      0.95      0.97      1838
weighted avg       0.99      0.99      0.99      1838



In [47]:
from sklearn import metrics

In [48]:
metrics.accuracy_score(y_test, pred)

0.9863982589771491

#### Testing

In [49]:
text_clf.predict(["Hi how are you doing?"])

array(['ham'], dtype=object)

In [50]:
text_clf.predict(["Congratulations! You have been selected to win our prize of $1 million"])

array(['spam'], dtype=object)